# Install useful libraries

In [ ]:
!wget https://raw.githubusercontent.com/Iamsdt/DLProjects/master/utils/Helper.py
!wget https://raw.githubusercontent.com/santurini/Ships-Classification-with-Neural-Networks-Ensemble/main/ShipUtils.py
!pip install timm
!pip install pytorch_pretrained_vit
!pip install GPUtil
!pip install einops

from IPython.display import clear_output
import torch
import torch.optim as optim
from torchvision import datasets, transforms, models, utils
import torchvision.transforms as T
from pytorch_pretrained_vit import ViT
from ShipUtils import *
import timm

clear_output()

# Parameters

In [4]:
num_classes = 7
batch_size = 8
num_workers = 2
epochs = 10 # the number of epochs

# Prepare loaders

In [6]:
# input_size = (224, 224)

transforms = [T.Compose([T.Resize(input_size), 
                              T.ToTensor(), 
                              T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))]),

              T.Compose([T.Resize(input_size), 
                         T.RandomHorizontalFlip(p = 1),
                         T.ToTensor(), 
                         T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))])]

train_loader, val_loader = get_loaders(path = './train_cleaned.csv', transform = transforms)

# Models training

In [ ]:
vit = ViT('B_16_imagenet1k', pretrained=True, num_classes = num_classes)
resnet = models.resnet152(pretrained=True); resnet.fc.in_features = num_classes
convnext = models.convnext_base(pretrained=True); convnext.classifier[2].in_features = num_classes
resnext = models.resnext101_32x8d(pretrained=True); resnext.fc.in_features = num_classes
se_resnext = timm.create_model('seresnext26d_32x4d', pretrained=True, num_classes=7)
xception = timm.create_model('xception', pretrained=True, num_classes=7)

## ViT B 16

In [ ]:
vit = train(vit, train_loader, val_loader, './vit')
vit = train(vit, train_loader, val_loader, './vit', 5, 'sgd') # 5 epoques with SGD

test(vit, './content/test', transforms[0], './vit.csv') # predict on test and save to csv

# Resnet152

In [ ]:
resnet = train(resnet, train_loader, val_loader, './resnet')
resnet = train(resnet, train_loader, val_loader, './resnet', 5, 'sgd') # 5 epoques with SGD

test(resnet, './content/test', transforms[0], './resnet.csv') # predict on test and save to csv

# ConvNeXt

In [ ]:
convnext = train(convnext, train_loader, val_loader, './convnext')
convnext = train(convnext, train_loader, val_loader, './convnext', 5, 'sgd') # 5 epoques with SGD

test(convnext, './content/test', transforms[0], './convnext.csv') # predict on test and save to csv

# ResNeXt

In [ ]:
resnext = train(resnext, train_loader, val_loader, './resnext')
resnext = train(resnext, train_loader, val_loader, './resnext', 5, 'sgd') # 5 epoques with SGD

test(resnext, './content/test', transforms[0], './resnext.csv') # predict on test and save to csv

# SE-ResNeXt

In [ ]:
se_resnext = train(se_resnext, train_loader, val_loader, './seresnext')
se_resnext = train(se_resnext, train_loader, val_loader, './seresnext', 5, 'sgd') # 5 epoques with SGD

test(se_resnext, './content/test', transforms[0], './seresnext.csv') # predict on test and save to csv

# Xception

In [ ]:
xception = train(xception, train_loader, val_loader, './xception')
xception = train(xception, train_loader, val_loader, './xception', 5, 'sgd') # 5 epoques with SGD

test(xception, './content/test', transforms[0], './xception.csv') # predict on test and save to csv

# Ensemble

In [3]:
models = ['vit', 'resnet', 'convnext', 'resnext', 'seresnext', 'xception']
df_compare, df_final = ensemble(models, './ensemble.csv')

['vit', 'resnet', 'convnext', 'resnext', 'seresnext', 'xception']